# COMP4471 Proj

- ZHANG, Yujun yzhanglo@connect.ust.hk

Group project of COMP4471@HKUST in 2022 Fall.

# 1 Environment Configuration

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    %cd '/content/drive/MyDrive/4471Group'
    import comp4471
    %cd '-'
except:
    import comp4471

%pwd
%ls

comp4471/    experiments/ main.ipynb


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import torch
from tensorboardX import SummaryWriter

# easier to print by putting variable as a single line
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# make matplotlib figures appear inline in the notebook rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Add some convenience functions to Pandas DataFrame.
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.2f}'.format
def mask(df, key, function):
  """Returns a filtered dataframe, by applying function to key"""
  return df[function(df[key])]

def flatten_cols(df):
  df.columns = [' '.join(col).strip() for col in df.columns.values]
  return df

pd.DataFrame.mask = mask
pd.DataFrame.flatten_cols = flatten_cols

# http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# 2 Data


## Data Loading

## Data Exploration

# 3 Preliminaries

There's no much symbol to be defined at this stage.

# 4 Models Definition

## Sanity Check by Overfitting on Small Data

## Training

In [2]:
import time
i = 0

In [8]:
while True:
    print(i)
    time.sleep(1)
    i += 1

5
6
7


KeyboardInterrupt: 

In [5]:
print(i)

4


# 7 Credit and Reference
